# Profitable App Profiles for the App Store and Google Play Markets

A Mobile Publishing company has requested assistence with the development of their latest app.  This organization is only interested in developing free apps, and the majority of their revenue comes from in-app ads.  In order to maximize their ad revenue, I need to identify traits and patterns that lead to high user engagement.

In order to answer this question, I have access to several datapoints relating to the top ~7,000 apps on the Apple Store, and the top ~11,000 on the Google Play Store.  

To begin, I load in the two data sets and perform some simple exploration to get a feel for how the data is structured.

In [1]:
from csv import reader

opened_1 = open('AppleStore.csv', encoding="utf8")
opened_2 = open('googleplaystore.csv', encoding="utf8")
read_1 = reader(opened_1)
read_2 = reader(opened_2)
data_apple = list(read_1)
data_google = list(read_2)

def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))
        

explore_data(data_apple, 0, 3)

explore_data(data_google, 0, 3)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15

## Data Hygiene

Before I can perform any meaningful analysis, the data needs to be cleaned and organized to ensure that it is accurate and digestible.  If my input is messy, my output will be messy regardless of the method I use to get it.

To align with the specific goals of this project, the data cleaning process will consist of four steps:

1. Correcting or removing innacurate data.
2. Detecting and removing duplicate data.
3. Removing non-English apps.
4. Removing apps that aren't free.

### Step 1: Correcting or removing inaccurate data.

In order to identify inaccurate data, I have consulted the discussion boards relating to the [Google Play](https://www.kaggle.com/lava18/google-play-store-apps/undefined) and [Apple Store](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/undefined) datasets.  I have decided to tackle the Google Play dataset first.

The first issue I discovered is that the app entry at index 10473 is missing the catagory entry that should be at `index[1]`.  Without this datapoint, the indexes of all entires that come after the category column will be offset - therefore causing any analysis done on this line to be innacurate at best, and impossible at worst.  to avoid any issues down the line I have elected to remove it from the dataset.



In [2]:
explore_data(data_google, 10471, 10475)
del data_google[10473]

['Jazz Wi-Fi', 'COMMUNICATION', '3.4', '49', '4.0M', '10,000+', 'Free', '0', 'Everyone', 'Communication', 'February 10, 2017', '0.1', '2.3 and up']


['Xposed Wi-Fi-Pwd', 'PERSONALIZATION', '3.5', '1042', '404k', '100,000+', 'Free', '0', 'Everyone', 'Personalization', 'August 5, 2014', '3.0.0', '4.0.3 and up']


['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


['osmino Wi-Fi: free WiFi', 'TOOLS', '4.2', '134203', '4.1M', '10,000,000+', 'Free', '0', 'Everyone', 'Tools', 'August 7, 2018', '6.06.14', '4.4 and up']




Now I can print the same index range and see that the "Life Made Wi-fi" entry with the missing column has been removed

In [3]:
explore_data(data_google, 10471, 10475)

['Jazz Wi-Fi', 'COMMUNICATION', '3.4', '49', '4.0M', '10,000+', 'Free', '0', 'Everyone', 'Communication', 'February 10, 2017', '0.1', '2.3 and up']


['Xposed Wi-Fi-Pwd', 'PERSONALIZATION', '3.5', '1042', '404k', '100,000+', 'Free', '0', 'Everyone', 'Personalization', 'August 5, 2014', '3.0.0', '4.0.3 and up']


['osmino Wi-Fi: free WiFi', 'TOOLS', '4.2', '134203', '4.1M', '10,000,000+', 'Free', '0', 'Everyone', 'Tools', 'August 7, 2018', '6.06.14', '4.4 and up']


['Sat-Fi Voice', 'COMMUNICATION', '3.4', '37', '14M', '1,000+', 'Free', '0', 'Everyone', 'Communication', 'November 21, 2014', '2.2.1.5', '2.2 and up']




The discussion board called out one more issue with the Google Play dataset - the app 'Command & Conquer: Rivals' has a type of 'NaN'.  As the price is listed as zero, the expected value is 'Free'.  

Below I show the issue, ensure it isnt repeated elsewhere, then replace the type value in question with 'Free' to keep any type analysis consistent.

In [4]:
print(data_google[0])
print()
for row in data_google:
    if row[0] == 'Command & Conquer: Rivals':
        print(row)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']

['Command & Conquer: Rivals', 'FAMILY', 'NaN', '0', 'Varies with device', '0', 'NaN', '0', 'Everyone 10+', 'Strategy', 'June 28, 2018', 'Varies with device', 'Varies with device']


In [5]:
for row in data_google:
    if row[6] == 'NaN':
        print(row)
        print()
        print(data_google.index(row))

['Command & Conquer: Rivals', 'FAMILY', 'NaN', '0', 'Varies with device', '0', 'NaN', '0', 'Everyone 10+', 'Strategy', 'June 28, 2018', 'Varies with device', 'Varies with device']

9149


In [6]:
data_google[9149][6] = 'Free'
print(data_google[9149])

['Command & Conquer: Rivals', 'FAMILY', 'NaN', '0', 'Varies with device', '0', 'Free', '0', 'Everyone 10+', 'Strategy', 'June 28, 2018', 'Varies with device', 'Varies with device']


After reviewing the discussion boards for the Apple Dataset, it appears that the only additional issues are with duplication which will be handled in the next step.

### Step 2: Detecting and removing duplicate data.

The next step is removing any duplicate entires from the datasets.  Counting the same app multiple times in my analysis will lead to schewed results, so I need to remove these duplicate entires before begining my calculations.

In [7]:
for row in data_google:
    if row[0] == 'Instagram':
        print(row)

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


The code above shows that Instagram is listed 4 times in the dataset.  The only difference between these entires is the number of user ratings, so I can deduce that the entry with the higest rating count is the most recent one.  The methodology I will use to dedupe these data sets, will be to keep the most recent entry for each app, as defined by the highest rating count.

Now to identify exactly how many duplicates there are in each dataset, as well as how long the datasets should be (excluding the header row) after they have been deduped.

In [8]:
apple_unique = []
apple_dupe = []
google_unique = []
google_dupe = []

for row in data_apple:
    name = row[1]
    if name in apple_unique:
        apple_dupe.append(name)
    else:
        apple_unique.append(name)

for row in data_google:
    name = row[0]
    if name in google_unique:
        google_dupe.append(name)
    else:
        google_unique.append(name)
        
print('number of duplicate enties in apple dataset: ' + str(len(apple_dupe)))
print('number of duplicate enties in google dataset: ' + str(len(google_dupe)))


# Subtracting one more from each of these values to account for the header row that will not be present in the cleaned data
print('Expected length of deduped apple dataset: ' + str(len(data_apple) - len(apple_dupe) - 1))
print('Expected length of deduped google dataset: ' + str(len(data_google) - len(google_dupe) - 1))

number of duplicate enties in apple dataset: 2
number of duplicate enties in google dataset: 1181
Expected length of deduped apple dataset: 7195
Expected length of deduped google dataset: 9659


#### Dataset Deduplication

In the code below, I have deduplicated the datasets using the higest number of ratings as the criteria that defines which duplicate entry is kept.  This was done using the steps below:

1. Created a dictionary to store the highest rating count for each unique app title in the google dataset and the apple dataset.

2. Created new lists called `android_clean` and `apple_clean` to capture the deduped datasets.

3. Populated these new lists using a single entry for each unique app title by:
    1. Checking that the rating count of the entry matches the max rating count stored in the dictionary.
    2. Checking that the app title has not already been added to the new list.
4. Checked the length of this new dataset to make sure it matches the expected value of 9659


In [9]:
reviews_max_google = {}

for row in data_google[1:]:
    name = row[0]
    n_reviews = float(row[3])
    if name in reviews_max_google and reviews_max_google[name] < n_reviews:
        reviews_max_google[name] = n_reviews
    elif name not in reviews_max_google:
        reviews_max_google[name] = n_reviews
                
android_clean = []
already_added = []

for row in data_google[1:]:
    name = row[0]
    n_reviews = float(row[3])
    if n_reviews == reviews_max_google[name] and name not in already_added:
        android_clean.append(row)
        already_added.append(name)

print('length of the android_clean dataset: ' + str(len(android_clean)))


length of the android_clean dataset: 9659


In [10]:
reviews_max_apple = {}

for row in data_apple[1:]:
    name = row[1]
    n_reviews = float(row[5])
    if name in reviews_max_apple and reviews_max_apple[name] < n_reviews:
        reviews_max_apple[name] = n_reviews
    elif name not in reviews_max_apple:
        reviews_max_apple[name] = n_reviews
                
apple_clean = []
already_added = []

for row in data_apple[1:]:
    name = row[1]
    n_reviews = float(row[5])
    if n_reviews == reviews_max_apple[name] and name not in already_added:
        apple_clean.append(row)
        already_added.append(name)

print('length of the apple_clean dataset: ' + str(len(apple_clean)))

length of the apple_clean dataset: 7195


### Step 3: Removing non-English apps.

Next on the data hygeine list is to remove all apps that are targeted towards non-english speaking audiences.  I am defining non-english apps as any that have more than 3 characters in the title that fall outside of the 0 - 127 ASCII range that is normally used to define english characters. 

The reason I am allowing up to three special characters, is that many app titles use characters like Emojis or Copyright symbols which also fall outside the 0 - 127 range.  Granted this will not be a perfect system and some apps will be classified incorrectly, but it should be close enough to serve my purposes.

In [11]:
def check_english(x):
    strx = str(x)
    counter = 0
    for i in strx:
        if ord(i) > 127:
            counter += 1      
    if counter > 3:
        return False
    else:
        return True

apple_eng = []
android_eng = []

for row in apple_clean:
    name = row[1]
    if check_english(name):
        apple_eng.append(row)

for row in android_clean:
    name = row[0]
    if check_english(name):
        android_eng.append(row)
        
print('We have ' + str(len(android_eng)) + ' Android apps remaining')
print('We have ' + str(len(apple_eng)) + ' Apple apps remaining')

We have 9614 Android apps remaining
We have 6181 Apple apps remaining


### Step 4: Removing apps that aren't free.

Lastly, we now need to remove any paid apps from our dataset.  As the client is only interested in developing free apps, including paid apps in my analysis will result in irrelevant or misleading results.  

In [12]:
apple_free = []
android_free = []

for row in apple_eng:
    price = row[4]
    if price == '0.0':
        apple_free.append(row)
        
for row in android_eng:
    price = row[7]
    if price == '0':
        android_free.append(row)
        
print('We have ' + str(len(android_free)) + ' Android apps remaining')
print('We have ' + str(len(apple_free)) + ' Apple apps remaining')

We have 8864 Android apps remaining
We have 3220 Apple apps remaining


## Data Analysis

Now that I have isolated only the apps that are relevant, the next step is structuring the data in a manner that will allow me to answer the question.  The go forward plan for development is outlined below:

1. Build a minimal Android version of the app, and add it to Google Play.
2. If the app has a good response from users, we develop it further.
3. If the app is profitable after six months, we build an iOS version of the app and add it to the App Store.

To align with this plan, I need to find app profiles that are successful in both markets.  The code below willl build frequency tables for the prime_genre column in the apple dataset, as well as the Category and Genre ones for the Android one.

In [13]:
def freq_table(dataset, index):
    freq_dict = {}
    for row in dataset:
        value = row[index]
        if value not in freq_dict:
            freq_dict[value] = 1
        else:
            freq_dict[value] +=1
            
    length = len(dataset)
    freq_dict_perc = {}
    
    for key in freq_dict:
        entry = freq_dict[key]
        perc = (entry/length) * 100
        freq_dict_perc[key] = perc
        
    return freq_dict_perc
        
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])



### Apple Dataset

The table below shows that Games dominate the free apps targeted towards English speakers on the app store with roughly 58% of the apps in question being games.  The next two categories by number of apps are Entertainment with just below 8% of the total, then Photo & Video with 5%.

This strongly implies that where free apps are concerned, entertainment apps tend to be more popular than practical ones with English speaking apple store users.  

While this alone doesn't necessarily mean that Games, Entertainment, and Photo and Video apps are more successful than other types of apps, it does show that the demand for these app genres can support a large number of products.  However, it also shows that competition is VERY high in these markets so differentiating factors and unique fucntionality are extra important where these types of apps are concerned.

On the other end of the spectrum, Business, Book, Navigation, Medical, and Catalogs are at the bottom of the list, all with less than 1% of the total dataset coming from these genres.  

This could mean one of several things:

1. Fewer users are interested in these types of apps.
2. There are undisputed market leaders for these audiences that have discouraged competition.
3. The market has not yet been taken advantage of and the competitive barrier to entry is low.

In [14]:
display_table(apple_free, 11)

Games : 58.13664596273293
Entertainment : 7.888198757763975
Photo & Video : 4.968944099378882
Education : 3.6645962732919255
Social Networking : 3.291925465838509
Shopping : 2.608695652173913
Utilities : 2.515527950310559
Sports : 2.142857142857143
Music : 2.049689440993789
Health & Fitness : 2.018633540372671
Productivity : 1.7391304347826086
Lifestyle : 1.5838509316770186
News : 1.3354037267080745
Travel : 1.2422360248447204
Finance : 1.1180124223602486
Weather : 0.8695652173913043
Food & Drink : 0.8074534161490683
Reference : 0.5590062111801243
Business : 0.5279503105590062
Book : 0.43478260869565216
Navigation : 0.18633540372670807
Medical : 0.18633540372670807
Catalogs : 0.12422360248447205


### Google Dataset

The google dataset contained two different columns that could be helpful in answering this question, so the anaylsis for both are below.

#### Catagory

The top the entries from the Category column are Family at 18.9%, Game at 9.7% and toosl at 8.5%.  Family apps probably include quite a few games, but the fact that tools ranks so high shows an interesting difference between the Apple and Google audiences.  I don't want to infer too much as there are several reasons that the numbers may be showing this way, but this initial overview suggests that the Google Play audience is more interested in practical apps than the Apple audience, as a larger percentage of the apps on the Google Play store cater to these types of users.

Business and Finance apps also make up a much larger percentage of free apps on the Google Play store, which also suggests that the audience may be older and more practically minded than that of the Apple Store.

In [15]:
display_table(android_free, 1)

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

#### Genre

The main difference I see between the Genre column and the Category one above, is that several categories are broken up into smaller sub categories in this set.  The two obvious ones are Family and Games, which have both been broken up into several more specific categories.  

As a high level overview, the Category set is more helpful, but the more detailed information in this dataset may be useful once a higher level course of action has been chosen.

In [16]:
display_table(android_free, 9)

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

The next step is to move a step furter in the analysis.  After all, the fact that there are a lot of a specific type of app in the market doesnt necessarily mean that that type of app gets the most engagement.  

For the Apple dataset, I have constructed a table that shows the average number of reviews for each type of app.  Number of reviews is not a perfect measurement, but as we don't have a number of downloads column for this dataset it is the closest we can get with the information at our disposal.

For the Google dataset, I have access to the number of installs per app which seems like a better unit of measurement than reviews, but unfortuantely these numbers are given as aproxomite counts instead of exact ones.  However, since I only care about the relative numbers and not the exact ones, the analysis will still be valid.

In [17]:
table = freq_table(apple_free, 11)
unsorted = []

for i in table:
    total = 0
    len_genre = 0
    for row in apple_free:
        genre_app = row[11]
        if genre_app == i:
            ratings = float(row[5])
            total += ratings
            len_genre += 1
    average_num = total / len_genre
    average_tup = (average_num, i)
    unsorted.append(average_tup)
    
sorted_table = sorted(unsorted, reverse = True)

for i in sorted_table:
    print(i[1] +' : ' + str(i[0]))

    



Navigation : 86090.33333333333
Reference : 74942.11111111111
Social Networking : 71548.34905660378
Music : 57326.530303030304
Weather : 52279.892857142855
Book : 39758.5
Food & Drink : 33333.92307692308
Finance : 31467.944444444445
Photo & Video : 28441.54375
Travel : 28243.8
Shopping : 26919.690476190477
Health & Fitness : 23298.015384615384
Sports : 23008.898550724636
Games : 22812.92467948718
News : 21248.023255813954
Productivity : 21028.410714285714
Utilities : 18684.456790123455
Lifestyle : 16485.764705882353
Entertainment : 14029.830708661417
Business : 7491.117647058823
Education : 7003.983050847458
Catalogs : 4004.0
Medical : 612.0


In [18]:
table = freq_table(android_free, 1)
unsorted = []
    
for i in table:
    total = 0
    len_category = 0
    for row in android_free:
        category_app = row[1]
        if category_app == i:
            installs = row[5]
            no_plus_installs = installs.replace('+', '')
            updated_installs = no_plus_installs.replace(',', '')
            total += float(updated_installs)
            len_category += 1
    average_num = total / len_category
    average_tup = (average_num, i)
    unsorted.append(average_tup)
    
sorted_table = sorted(unsorted, reverse = True)

for i in sorted_table:
    print(i[1] +' : ' + str(i[0]))
    
            
        

COMMUNICATION : 38456119.167247385
VIDEO_PLAYERS : 24727872.452830188
SOCIAL : 23253652.127118643
PHOTOGRAPHY : 17840110.40229885
PRODUCTIVITY : 16787331.344927534
GAME : 15588015.603248259
TRAVEL_AND_LOCAL : 13984077.710144928
ENTERTAINMENT : 11640705.88235294
TOOLS : 10801391.298666667
NEWS_AND_MAGAZINES : 9549178.467741935
BOOKS_AND_REFERENCE : 8767811.894736841
SHOPPING : 7036877.311557789
PERSONALIZATION : 5201482.6122448975
WEATHER : 5074486.197183099
HEALTH_AND_FITNESS : 4188821.9853479853
MAPS_AND_NAVIGATION : 4056941.7741935486
FAMILY : 3695641.8198090694
SPORTS : 3638640.1428571427
ART_AND_DESIGN : 1986335.0877192982
FOOD_AND_DRINK : 1924897.7363636363
EDUCATION : 1833495.145631068
BUSINESS : 1712290.1474201474
LIFESTYLE : 1437816.2687861272
FINANCE : 1387692.475609756
HOUSE_AND_HOME : 1331540.5616438356
DATING : 854028.8303030303
COMICS : 817657.2727272727
AUTO_AND_VEHICLES : 647317.8170731707
LIBRARIES_AND_DEMO : 638503.734939759
PARENTING : 542603.6206896552
BEAUTY : 51315

## Recommendations

Based on the data, I recommend developing a reference app to put on the Google Play and Apple Stores.  Reference has the second highest level of average user engagement among english free apps on the Apple store, and while the insights from the Google Play store are not as conclusive - the data shows that the audience is there to support a multiple stage rollout across the two platforms.  Additionally, building a competative product in this space will likely be easier than trying to compete with the likes of Apple Maps, Spotify, and Instagram in the other high user engagement Genres.

The other big plus to moving in this direction, is that we won't have to deal with the same level of complexity that we would with the other top audiences.  Breaking into Navigation, Weather, or Finance would likely entail having to connect to complex and expensive APIs, Music or Book apps would require us to aquire IP rights (and the ads on these types of apps are usually viewed as invasive by users), and the popular food and drink apps are primarly owned by restauraunt chains which is out of the scope of our orgnaization.

While reference isnt very sexy, building a highly functional dictionary, thesaurus, or translation app may be the fastest way to building a large, loyal and engaged audience to support the organization's ad revenues.
